### 1. Few shot Learning

소량의 학습만으로 새로운 작업을 학습하거나 일반화하는 것을 의미한다. 모델에 어떻게 반응하기를 원하는지에 대한 예시를 제공하는 것으로, 이 기술은 전체 아키텍처를 건드리지 않고도 모델 맞춤화를 가능하게 한다.

In [1]:
import numpy as np
import pandas as pd
import openai
from openai import OpenAI
import os

with open('../../config/api.key') as file :
    lines = file.readlines()
    api_key = lines[0].strip()
    serp_api_key = lines[1].strip()
    langsmith_api_key = lines[2].strip()

openai.api_key = api_key

In [2]:
system_message = """
당신은 AI 마케팅 도우미입니다. 사용자가 새로운 제품 이름에 대한 캐치프레이즈를 만들 수 있도록 도와줍니다.
주어진 제품명에 대해 다음 예시와 비슷한 홍보 문구를 만들어주세요.

Apple - Think Different
Samsung - Do What You Can’t
SK 텔레콤 - 또 다른 세상을 만날 땐 잠시 꺼두셔도 좋습니다.
Amazon - Earth’s Most Customer-Centric Company
Microsoft - Cloud First
Google - Don’t Be Evil

제품명 : 
"""

product_name = 'iPhone'

In [3]:
client = OpenAI(api_key = openai.api_key)

response = client.chat.completions.create(
    model = 'gpt-4o-mini',
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': product_name},
    ]
)

In [4]:
print(response.choices[0].message.content)

iPhone - 혁신을 손안에!


대부분의 경우, fewshot learning은 fine tuning을 적절한 도구로 생각할 수 있는 극도로 특수한 시나리오에서도 모델을 맞춤화할 수 있을 만큼 강력하다는 점을 명심하자. 실제로 적절한 few shot learning은 fine tuning process만큼 효과적일 수 있다. 다른 예를 살펴보자. 감정 분석에 특화된 모델을 개발하고자 한다고 가정해 보자. 이를 위해 긍정 또는 부정 등 원하는 결과와 함께 다양한 감정을 가진 일련의 텍스트 예시를 제공한다. 이 예제 세트는 지도 학습 과업을 위한 작은 훈련 세트에 불과하며, fine tuning과 유일한 차이점은 모델의 parameter를 업데이트 하지 않는다는 점이다.

In [5]:
system_message = """
당신은 금융, 경제에 종사하고 있는 사람들의 연설을 전문적으로 분석하는 분석가입니다. 
연설의 감정을 향후 주식시장에 미칠 영향을 기준으로 positive, negative 로 분류합니다.

원문이 한국어가 아닌 경우, 한국어로 번역을 하되 번역된 원문은 제공하지 마세요.

다음 텍스트들을 예시로 사용할 수 있습니다.

텍스트 : "I love this product! It's fantastic and works perfectly."
Positive

텍스트 : "I'm really disappointed with the quality of the food."
Negative

텍스트 : "This is the best day of my life."
Positive

텍스트 : "I can't stand the noise in this restaurant."
Negative

감정만을 구두점 없이 출력합니다.

텍스트 : 
"""

분류기를 테스트하기 위해 Kaggle에서 제공되는 영화 리뷰의 IMDB 데이터베이스를 사용한다.

In [6]:
df = pd.read_csv('../../data/IMDB Dataset.csv', encoding = 'utf-8')
df['sentiment'] = df['sentiment'].replace(
    {0: 'negative', 1: 'positive'}
)
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [7]:
df = df.sample(n=10, random_state=42)

In [8]:
df

,review,sentiment
33553,I really liked this Summerslam due to the look...,positive
9427,Not many television shows appeal to quite as m...,positive
199,The film quickly gets to a major chase scene w...,negative
12447,Jane Austen would definitely approve of this o...,positive
39489,Expectations were somewhat high for me when I ...,negative
42724,I've watched this movie on a fairly regular ba...,positive
10822,For once a story of hope highlighted over the ...,positive
49498,"Okay, I didn't get the Purgatory thing the fir...",positive
4144,I was very disappointed with this series. It h...,negative
36958,The first 30 minutes of Tinseltown had my fing...,negative


In [9]:
def process_text(text) : 
    response = client.chat.completions.create(
        model = 'gpt-4o-mini',
        messages = [
            {'role': 'system', 'content': system_message},
            {'role': 'user', 'content': text},
        ]
    )
    return response.choices[0].message.content

df['predicted'] = df['review'].apply(process_text)

In [10]:
df

,review,sentiment,predicted
33553,I really liked this Summerslam due to the look...,positive,Negative
9427,Not many television shows appeal to quite as m...,positive,Positive
199,The film quickly gets to a major chase scene w...,negative,Negative
12447,Jane Austen would definitely approve of this o...,positive,Positive
39489,Expectations were somewhat high for me when I ...,negative,Negative
42724,I've watched this movie on a fairly regular ba...,positive,Positive
10822,For once a story of hope highlighted over the ...,positive,Positive
49498,"Okay, I didn't get the Purgatory thing the fir...",positive,Negative
4144,I was very disappointed with this series. It h...,negative,Negative
36958,The first 30 minutes of Tinseltown had my fing...,negative,Negative


label과 predicted 열을 비교한 결과를 보면 알 수 있듯이, 모델을 fine tuning하지 않고도 모든 리뷰를 정확하게 분류할 수 있다. 이것은 모델 전문화 측면에서 few shot learning 기술을 통해 달성할 수 있는 것의 한 예일 뿐이다.

### 2. Chain of Thought(CoT)

사고 연쇄(Chain of Thought)는 중간 추론 단계를 통해 복잡한 추론을 가능하게 하는 기법이다. 또한 모델이 자신의 추론을 설명하도록 유도하여 너무 빠르지 않도록 '강제'함으로써 잘못된 응답을 제공할 위험을 방지한다. LLM이 일반적인 일차 방정식을 풀게 하고 싶다고 가정하자. 이를 위해 LLM이 따라야 할 기본 추론 목록을 제공한다.

In [11]:
system_message = """
일반적인 1차 방정식을 해결하려면 다음 단계를 따르세요.

1. 방정식 식별 : 해결하려는 방정식을 식별합니다.
   방정식은 'ax + b = c' 형태여야 합니다.
   여기서 'a'는 변수의 계수, 'x'는 변수, 'b'는 상수, 'c'는 또 다른 상수입니다.
2. 변수 고립화 : 목표는 변수 'x'를 방정식 한쪽에 고립시키는 것입니다.
   이를 위해 다음 단계를 수행합니다:
   a. 상수 추가 또는 빼기 : 상수를 한쪽으로 이동시키기 위해 양쪽에서 'b'를 더하거나 뺍니다.
   b. 계수로 나누기 : 'x'를 고립시키기 위해 양쪽을 'a'로 나눕니다.
      'a'가 0이면 방정식에는 고유한 해가 없을 수 있습니다.
3. 단순화 : 방정식의 양쪽을 최대한 단순화합니다.
4. 'x'에 대해 해결 : 'x'를 한쪽에 고립시키면 해결책을 얻을 수 있습니다.
   이는 'x = 값' 형태가 될 것입니다. 
5. 해 검토 : 찾은 'x' 값을 원래 방정식에 대입하여 방정식을 만족하는지 확인합니다.
   그렇다면 올바른 해결책을 찾은 것입니다.
6. 해 표현 : 해결책을 명확하고 간결한 형태로 작성합니다.
7. 특수 경우 고려 : 'a'가 0일 때 해가 없거나 무한히 많은 해가 있을 수 있는 특수한 경우를 인지합니다.

방정식 : 
"""

equation = "3x + 5 = 11"

response = client.chat.completions.create(
    model = 'gpt-4o-mini',
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': equation},
    ]
)

In [12]:
print(response.choices[0].message.content)

주어진 방정식은 \(3x + 5 = 11\)입니다. 이 방정식을 다음 단계에 따라 해결하겠습니다.

1. **방정식 식별**: \(3x + 5 = 11\)는 'ax + b = c' 형태입니다.
   - 여기서 \(a = 3\), \(b = 5\), \(c = 11\)입니다.

2. **변수 고립화**:
   a. **상수 추가 또는 빼기**:
   양쪽에서 5를 빼줍니다.
   \[
   3x + 5 - 5 = 11 - 5
   \]
   즉,
   \[
   3x = 6
   \]

   b. **계수로 나누기**:
   양쪽을 3으로 나눕니다.
   \[
   x = \frac{6}{3}
   \]
   즉,
   \[
   x = 2
   \]

3. **단순화**: 이미 단순화된 상태입니다.

4. **'x'에 대해 해결**: \(x = 2\)입니다.

5. **해 검토**: \(x = 2\)을 원래 방정식에 대입해 봅니다.
   \[
   3(2) + 5 = 11 \implies 6 + 5 = 11 \implies 11 = 11
   \]
   따라서, 방정식을 만족합니다.

6. **해 표현**: 해결책은 \(x = 2\)입니다.

7. **특수 경우 고려**: 여기서 \(a = 3\)은 0이 아니므로, 특별한 경우는 없습니다. 

따라서, 주어진 방정식 \(3x + 5 = 11\)의 해는 \(x = 2\)입니다.


모델은 meta prompt에 지정된 7단계를 명확하게 따랐으며, 이를 통해 이 과업을 수행하는 데 '시간적 여유'를 가질 수 있다. 응답하기 전에 추론이 필요한 더 복잡한 과업에서 더 나은 결과를 얻기 위해 이 기능을 단답형 prompt와 결합할 수도 있다.

CoT를 사용하면 모델에 중간 추론 단계를 생성하도록 유도한다. 이는 다음 절에서 살펴볼 또 다른 추론 기법의 구성 요소이기도 하다.

### 3. Reason and Act (ReAct)

Shunyu Yao 등의 논문 ReAct : Synergizing Reasoning and Acting in Language Models ReAct에 소개된 ReAct는 추론과 행동을 LLM과 결합하는 일반적 패러다임이다. ReAct는 언어 모델에 과업에 대한 언어적 추론 추적과 행동을 생성하도록 유도하고 웹 검색이나 DB와 같은 외부 소스에서 정보를 얻기도 한다. 이를 통해 언어 모델은 동적 추론을 수행하고, 외부 정보를 기반으로 행동 계획을 신속하게 조정할 수 있다. 예를 들어, 먼저 질문에 대해 추론한 다음 웹에 질의를 보내는 행동을 수행하고 검색 결과에서 정보를 수신한 다음 결론에 도달할 때까지 이러한 생각, 행동, 관찰 루프를 계속해 언어 모델에 질문에 대한 답변을 유도할 수 있다. CoT와 ReAct 접근 방식의 차이점은, CoT는 언어 모델에 과업에 대한 중간 추론 단계를 생성하라는 메세지를 표시하는 반면, ReAct는 언어 모델에 과업에 대한 중간 추론 단계, 행동 및 관찰을 생성하라는 메세지를 표시한다는 점이다.

'행동' 단계는 일반적으로 웹 검색과 같은 외부 도구와 상호 작용할 수 있는 LLM의 가능성과 관련이 있다. 예를 들어, 비교적 최근에 주식시장이 하락한 이유에 대한 정보를 모델에 요청하고 싶다고 가정하자. 이를 위해 SerpAPI 도구를 활용해 똑똑한 langchain Agent를 구축하고자 한다. 아래 예시를 보고, 어떤식으로 구현할 수 있는지 살펴보도록 한다. 아직 Agent 모델에 대해서는 진행하지 않았으므로, 어떤 식으로 진행이 되는지 간단히 살펴본다.

dependency : `langchain-community`, `serpapi`, `google-search-results`

In [13]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent, load_tools
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(
    model_name = 'gpt-4o-mini',
    api_key = openai.api_key,
    
)
tools = load_tools(
    ['serpapi'],
    serpapi_api_key = serp_api_key
)
prompt = hub.pull('hwchase17/react')

agent = create_react_agent(chat, tools, prompt)
agent_executor = AgentExecutor(
    agent = agent,
    tools = tools,
    verbose = True
)

/Users/junghunlee/anaconda3/envs/FinancialMachineLearning/lib/python3.11/site-packages/langsmith/client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


검색을 위해 langchain에서는 ReAct와 관련된 에이전트가 미리 생성되어 있다.

In [14]:
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


실제로 사용한 결과는 다음과 같다. input을 프롬프트 형식으로 유지한다면, 양방향 소통이 가능한 에이전트를 생성할 수 있다.

In [15]:
result = agent_executor.invoke(
    {'input': '2024년 8월 5일 미국의 주식시장이 갑자기 하락한 이유는?'}
)
print(result)



> Entering new AgentExecutor chain...
2024년 8월 5일 미국의 주식시장이 하락한 이유를 알아보기 위해 최근 뉴스를 검색해야겠다.  
Action: Search  
Action Input: "2024년 8월 5일 미국 주식시장 하락 이유"  ['미국발 경기침체 우려와 주요국 통화정책 변화, 중동 분쟁과 미국 대통령 선거 등 대외적 불확실성이 해소되지 않은 까닭에 증권시장에선 당분간 변동성이 ...', '증권사들의 프로그램 매매와 미국 경제의 불안이 이유였다고 하지만 정확한 이유는 아닌 것으로 밝혀졌다. 블랙먼데이에 미국 시장 대형주인 IBM, 엑손( ...', '5일 미국발 경기 침체 우려로 국내 증시를 포함한 아시아 증시가 급락한 가운데 미국 나스닥100 선물 지수 역시 급락 중이다.', '이번 주식시장의 폭락은 8월 2일 미국 노동부가 7월 실업률이 4.3%였다는 발표로 촉발된 불황의 공포, 이른바 R의 공포가 도화선이었다는 것이 중론이다.', '이번 주(8월 5일~8월 9일) 뉴욕증시는 미국의 경기 침체 우려 속 높은 변동성을 보일 것으로 전망된다.지난주 뉴욕증시는 여러 대형 이벤트 속 롤러 ...', '미국채 금리는 일제히 하락. 시장 예상을 대폭 하회한 비농업 고용 및 실업률 상승 등의 지표영향을 받았으며 2년물 금리가 26.8bp 내렸고 10년물은 ...', '5일 미국 경기 침체에 대한 우려 등으로 한국 증시가 폭락하며 최악의 하루를 경험했다. 이날 코스피는 전 거래일 대비 8.77% 내린 2,441.55, 코스닥은 ...', '미국 경기 침체 우려, 외국인 투자자의 대량 매도, 엔 캐리 트레이드 청산 우려, 인공지능 관련 주식의 수익성 우려, 중동 지정학적 불안 등이 주요 원인 ...']2024년 8월 5일 미국 주식시장이 하락한 이유는 미국발 경기 침체 우려와 관련된 여러 요소가 복합적으로 작용했기 때문이다. 특히, 8월 2일 발표된 미국의 7월 실업률이 4.3%로 증가하면서 불황에 대한 공포, 즉 'R의 공

이제 더욱 세밀한 문제에 대해 질문하고 중간 단계를 살펴봄으로써 에이전트를 시험해 보자.

In [16]:
result = agent_executor.invoke(
    {'input': '2025년 1월 금리 인하가 되지 않을 것이란 우려가 확산되면서 시장의 변동성이 커졌는데, FOMC 이후에 주식 시장이 하락할까? 그에 대한 이유는?'}
)
print(result)



> Entering new AgentExecutor chain...
FOMC(연방공개시장위원회) 회의 이후 주식 시장의 하락 여부는 여러 요인에 의해 결정될 수 있다. 금리 인하에 대한 우려가 커지면서 투자자들은 불확실성을 느끼고 있으며, 이는 주식 시장에 부정적인 영향을 미칠 수 있다. 또한, FOMC에서의 발언이나 경제 지표 발표가 주식 시장에 어떤 영향을 미칠지도 고려해야 한다. 시장의 반응을 예측하기 위해 현재의 경제 상황과 FOMC의 최근 결정들을 분석하는 것이 필요하다.

Action: Search
Action Input: "FOMC 주식 시장 하락 금리 인하 우려 2025년 1월"
['올해 12월까지 8번의 연방공개시장위원회(FOMC) 회의에서 기준금리가 25bp 인상될 확률은 모두 0%를 유지하고 있다. 1월 금리동결 확률은 97.3%까지 ...', '연준 FOMC 내부에서 "금리인하 당분간 중단" 의견이 나오고 있다. 금리인하 중단 소식에 뉴욕증시에서 비트코인 리플 등이 "날벼락" 을 밪고 있다.', '이틀 전 연준이 올해 마지막 연방공개시장위원회(FOMC)에서 기준금리를 4.25~4.50%로 25bp 인하한 것을 감안하면 내년 두 차례, 많아도 세 차례 금리 인하 ...', '전반적으로 매파적이었습니다. 하지만 애초 12월 FOMC는 2025년 금리 인하 횟수를 2회로 제시하는 등 시장에 충격을 줄 정도로 매파적인 회의였습니다.', '... 1월 FOMC 금리 동결 가능성이 높아지고 추가금리 인하 가능성이 낮아지면서 10년 만기 금리는 급등했습니다. 이로 인하여 주식시장의 변동성이 급등 ...', '2025년에도 미 증시가 가장 믿음직한 시장이라는 데는 별 이견이 없다. 다만 최근 미 중앙은행 연방준비제도(Fed·연준)가 금리 인하 속도를 조절하겠다는 ...', '정리해 보도록 하자!! \u200b. \u200b. 2024년 12월 마지막 FOMC. 금리인하 전망 ... 2025년 1월 FOMC에서는. 금리 동결 가능성도 높게 점쳐지고 있

모델이 답을 도출하기까지 어떤 식으로 작동하였는지 주목하자. 이는 모델에 단계별로 생각하도록 유도하고, 추론의 각 단계를 명시적으로 정의하는 것이 어떻게 모델이 대답하기 전에 '더 현명하고' 더 주의하게 만드는지를 보여주는 좋은 예이다. 이는 또한 환각을 방지하는 훌륭한 기법이기도 하다.

전반적으로 prompt engineering은 강력한 분야로, 아직 초기 단계에 있지만 이미 LLM 기반의 애플리케이션에서 널리 채택되고 있다. 다음 장에서는 웹데이터를 수집해 투자자에게 유용한 정보를 제공하는 애플리케이션을 디자인 해 보도록 하자.